# 데이터 가져오기

In [73]:
import os
import tarfile
import urllib.request

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/rickiepark/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [4]:
fetch_housing_data()

In [5]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [6]:
housing = load_housing_data()
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


# 머신러닝 알고리즘을 위한 데이터 준비

In [72]:
housing = strat_train_set.drop("median_house_value", axis=1) # 훈련 세트를 위해 레이블 삭제
housing_labels = strat_train_set["median_house_value"].copy()

In [36]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat
4629,-118.30,34.07,18.0,3759.0,NaN,3296.0,1462.0,2.2708,<1H OCEAN,2
6068,-117.86,34.01,16.0,4632.0,NaN,3038.0,727.0,5.1762,<1H OCEAN,4
17923,-121.97,37.35,30.0,1955.0,NaN,999.0,386.0,4.6328,<1H OCEAN,4
13656,-117.30,34.05,6.0,2155.0,NaN,1039.0,391.0,1.6675,INLAND,2
19252,-122.79,38.48,7.0,6837.0,NaN,3468.0,1405.0,3.1662,<1H OCEAN,3


중간값이 수치형 특성에서만 계산될 수 있기 때문에 텍스트 특성을 삭제합니다:

In [42]:
housing_num = housing.drop("ocean_proximity", axis=1)
# 다른 방법: housing_num = housing.select_dtypes(include=[np.number])

**2.5.3 나만의 변환기**

 데이터의 특별한 정제작업, 데이터 특성들을 조합등의 작업을 위해 자신만의 변환기를 만들때가 필요합니다.

다음은 가구당 방의 비율, 가구당 인원비율을 변환 할수 있도록 하는 변환기 소스 입니다


In [58]:
from sklearn.base import BaseEstimator, TransformerMixin 

# 열 인덱스
rooms_ix, bedrooms_ix, population_ix, households_ix = 3, 4, 5, 6  # housing 데이터 열지정

class CombinedAttributesAdder(BaseEstimator, TransformerMixin): 
    def __init__(self, add_bedrooms_per_room=True): 
        self.add_bedrooms_per_room = add_bedrooms_per_room
        
    def fit(self, X, y=None):
        return self  # 아무것도 하지 않습니다

    def transform(self, X):
        rooms_per_household = X[:, rooms_ix] / X[:, households_ix]    # 가구당 방의 비율 방 개수 total_rooms 와 가구수 households 포함
        population_per_household = X[:, population_ix] / X[:, households_ix] # 가구당 인원비율 , 인구 population 과 가구수 households 포함

        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix] # 전체 방중 침실 비율
            
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.to_numpy())

# TransformerMixin 상속하면 fit, transform 메서드만 만들어도 자동으로 fit_transform 메소드를 자동으로 생성해준다
  # attr_adder.fit_transform()
# BaseEstimator 상속하면 하이퍼파라미터 튜닝에 필요한 get_params, set_params 추가로 생성
  # attr_adder.get_params()   
  # attr_adder.set_params()

책에서는 간단하게 인덱스 (3, 4, 5, 6)을 하드코딩했지만 다음처럼 동적으로 처리하는 것이 더 좋습니다:

In [59]:
col_names = "total_rooms", "total_bedrooms", "population", "households"
rooms_ix, bedrooms_ix, population_ix, households_ix = [
    housing.columns.get_loc(c) for c in col_names] # 열 인덱스 구하기

또한 `housing_extra_attribs`는 넘파이 배열이기 때문에 열 이름이 없습니다(안타깝지만 사이킷런을 사용할 때 생기는 문제입니다). `DataFrame`으로 복원하려면 다음과 같이 할 수 있습니다:

In [60]:
housing_extra_attribs = pd.DataFrame(
    housing_extra_attribs,
    columns=list(housing.columns)+["rooms_per_household", "population_per_household"],
    index=housing.index)
housing_extra_attribs.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity,income_cat,rooms_per_household,population_per_household
17606,-121.89,37.29,38,1568,351,710,339,2.7042,<1H OCEAN,2,4.62537,2.0944
18632,-121.93,37.05,14,679,108,306,113,6.4214,<1H OCEAN,5,6.00885,2.70796
14650,-117.2,32.77,31,1952,471,936,462,2.8621,NEAR OCEAN,2,4.22511,2.02597
3230,-119.61,36.31,25,1847,371,1460,353,1.8839,INLAND,2,5.23229,4.13598
3555,-118.59,34.23,17,6592,1525,4459,1463,3.0347,<1H OCEAN,3,4.50581,3.04785


2.5.4 특성 스케일

데이터에 적용할 가장 중요한 변환 중 하나가 특성 스케일링 입니다.

머신러닝 알고리즘은 대부분 입력 숫자 특성들의 스케일이 많이 다르면 잘 작동하지 않습니다.

min-max 스케일링과 표준화가 널리 사용됩니다.

min-max 스케일링은 0~1 범위에 들도록 값을 이동하고 스케일을 조정

데이터에서 최솟값을 뺀 후 최대값과 최소값의 차이로 나누면됨

사이키런의 minmaxScaler 변환기 사용

만약 0~1사이를 원하지 않는다면 feature_range 매개변수로 범위를 조정할 수 있음

표준화는 먼저 평균을 뺀 후 표준편차로 나누어 결과분포의 분산이 1이 되도락함

min-max스케일링과는 달리 표준화는 범위의 상한과 하한이 없어 알고리즘에 따라 문제를 야기할수 있지만 이상치에 영향을 덜 받는 특징이 있음
사이킷런에서는 표준화를 위한 StandardScaler 변환기 있음


- 축약

Min-max scailing : 0과 1사이의 값이 되도록 조정

표준화(standardization) : 평균이 0, 분산이 1이 되도록 만들어 줌 (Scikit-learn의 StandardScaler사용)







**- 2.5.5 변환 파이프라인**

수치형 특성을 전처리하기 위해 파이프라인을 만듭니다:

변환 단계가 많으며 정확한 순서대로 실행 되어야 함.

연속된 변환을 술서대로 처리할 수 있도록 도와주는 사이킷런에서는 파이프라인 클래스를 제공합니다.

In [61]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),  # 결측치 대체 함수   median 옵션으로 중간값으로 대채
        ('attribs_adder', CombinedAttributesAdder()), # 상단에 정의한 나만의 데이터 변환기
        ('std_scaler', StandardScaler()), # 표준화 
    ])


# SimpleImputer -> Transformer -> StandardScaler 순으로 진행
#마지막 단계를 제외한 모든 단계는 변환기 (fit(), fit_transform() 메소드를 포함 해야함).
#파이프라인의 fit() 메소드를 호출하면 모든 변환기의 fit_transform() method를 순서대로 호출, 한 단계의 출력을 다음 단계의 입력으로 전달. 마지막 단계에서는 fit() 메소드만 호출.

housing_num_tr = num_pipeline.fit_transform(housing_num)

In [62]:
housing_num_tr

array([[-1.15604281,  0.77194962,  0.74333089, ..., -0.31205452,
        -0.08649871,  0.15531753],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.21768338,
        -0.03353391, -0.83628902],
       [ 1.18684903, -1.34218285,  0.18664186, ..., -0.46531516,
        -0.09240499,  0.4222004 ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.3469342 ,
        -0.03055414, -0.52177644],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.02499488,
         0.06150916, -0.30340741],
       [-1.43579109,  0.99645926,  1.85670895, ..., -0.22852947,
        -0.09586294,  0.10180567]])

번주형 열과 수치형 열을 각각 다루었지만 

하나의 변환기로 각 열마다 적절한 변환을 적용하여 모든 열을 처리할 수 있도록 할 수 있습니다


In [63]:
from sklearn.compose import ColumnTransformer   # 하나의 변환기로 각 열마다 적절한 변환을 적용하여 모든 열을 처리할 수 있도록 하는 기능

num_attribs = list(housing_num) # 수치형 데이터 컬럼명
cat_attribs = ["ocean_proximity"] # 범주형데이터 컬럼명

full_pipeline = ColumnTransformer([
        ("num", num_pipeline, num_attribs), # 이름, 변환기, 변환기 적용될 열이름
        ("cat", OneHotEncoder(), cat_attribs),# 이름, 변환기, 변환기 적용될 열이름
    ])

housing_prepared = full_pipeline.fit_transform(housing)

In [64]:
housing_prepared

array([[-1.15604281,  0.77194962,  0.74333089, ...,  0.        ,
         0.        ,  0.        ],
       [-1.17602483,  0.6596948 , -1.1653172 , ...,  0.        ,
         0.        ,  0.        ],
       [ 1.18684903, -1.34218285,  0.18664186, ...,  0.        ,
         0.        ,  1.        ],
       ...,
       [ 1.58648943, -0.72478134, -1.56295222, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.78221312, -0.85106801,  0.18664186, ...,  0.        ,
         0.        ,  0.        ],
       [-1.43579109,  0.99645926,  1.85670895, ...,  0.        ,
         1.        ,  0.        ]])

In [65]:
housing_prepared.shape # 주어진 행렬 크기 출력

(16512, 17)